# Predicting swell

The docking jetty at Barrow Island

![barrow](images/barrow.png)

The waters around Barrow

![chart](images/barrow-chart.png)


The buoy location

![buoy](images/buoy.png)

# Data
* Wave parameters
    * Specific Wave Height
    * Period peak wave
    * Direction

* Spectra 2d Frequency vs Direction

![Spectra](images/2020-08-18Spectra.png)

![HS](images/2020-08-18HS.png)

From the spectra many parameters can be derived.

In [1]:
def getMetricsNODC(theta, freq, E, u10=None, depth=999.0, nan=999.0, fh=None):
    """
    Calculate standard mean wave parameters from directional
    spectral density following Rogers & Wang (2007). The
    routine uses WAVEWATCH III(R) spectral output.

    Reference:
       Tolman H (2009)  User manual and system documentation
           of WAVEWATCH III(R) version 3.14. Tech Note 276, 220p.
       Kuik AJ, GP van Vledder & L Holthuijsen (1988)
           JPO, 18(7), 1020-1034

    Input:
      theta - directions      (in radians, _constants.deg2rad*(90.-nc.variables['direction']))
      freq  - frequencies     (in Hz)
      E     - directional wave spectra (in m2/rad-Hz)
      u10   - wind speed      (in m/sec)
      depth - depth           (in m)
      nan   - value used for nan's (not a number)
      fh    - cut-off frequency for the tail (in Hz)
      basic - only calculate a subset of parameters
    Output:
      mPar   - mean parameters as type of python dictionary:
               mPar['FP']    peak frequency           --+
               mPar['FP1']   peak freq. (Young 1999)    |
               mPar['DP']    peak wave period           |
               mPar['DP1']   peak wave period w/ FP1    |
               mPar['DPD']   peak wave direction        |
               mPar['SWH']   significant wave height    |
               mPar['MWD']   mean wave direction        |> integral
               mPar['T01']   average period             |
               mPar['T02']   zero-crossing period       |
               mPar['TE']    energy period              |
               mPar['CGE']   wave energy flux           |
               mPar['BT']    breaking probability     --+
               mPar['SPRD']  mean directional spread               --+
               mPar['PSPR']  directional spread at FP                |
               mPar['PSP2']  directional spread at 2FP               |
               mPar['FSPR']  directional spread as func. of freq.    |
               mPar['AFP']   directional narrowness at FP            |> spectral
               mPar['A2FP']  direction narrowness at 2FP             |
               mPar['ALPHA'] equilibrium interval level (alpha)      |
               mPar['GAMMA'] spectral peakedness (gamma)             |
               mPar['NU']    spectral width (Longuet-Higgins 1975) --+
               mPar['WND']   wind speed               --+
                                                        |> other
                                                      --+
    """


In [2]:
# Data preparation

All the input files are in NetCDF.

Under the hood NetCDF uses HDF5, BUT (a big but) you should not read it from HDF5 as:
* Scaling is applied using metadata
* The metadata is stored in other tables - not HDF5 attributes

So I used xarray which opens then file and presents a dictionary back.

SyntaxError: invalid syntax (<ipython-input-2-236d8f9339b5>, line 1)

In [ ]:
def load_prediction_data(netcdf_filename, **kwargs):
    tag_list = ["times"]
    for label in kwargs["wave_parameters"]:
        tag_list.append(f"{label}")

    data = xr.open_dataset(netcdf_filename)
    return {tag: np.copy(data[tag]) for tag in tag_list}


## Input data

Each hindcast is for 7 days, with a forecast exevy hour, after 3 days the accuracy of the forecast is poor as can be seen below

![day1](images/day1.png)

![day2](images/day2.png)


So we use a sliding window to get N

Input data:
* 1D spectra in frequency
* 1D spectra in direction
* Wind speed and direction
* Tides
* Time of day

# Models

To be sure of a good answer I train 4 different models and then combined the results using an ensemble

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import (
    Activation,
    Layer,
    GRU,
    Dense,
    Bidirectional,
    concatenate,
    Conv1D,
    Flatten,
    BatchNormalization,
    Add,
    MaxPooling1D,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def cnn_residual_block(inputs_data: Layer, filters: int, kernel_size: int) -> Layer:
    x = BatchNormalization()(inputs_data)
    x = Activation("relu")(x)
    x = Conv1D(
        filters=filters, kernel_size=kernel_size, use_bias=False, padding="same"
    )(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv1D(filters=filters, kernel_size=kernel_size, use_bias=True, padding="same")(
        x
    )
    return Add()([x, inputs_data])


def dense_block(input_data: Layer, units: int, name: str = None) -> Layer:
    x = Dense(units, use_bias=False, name=name)(input_data)
    x = BatchNormalization()(x)
    return Activation("tanh", name=f"activation_{name}" if name is not None else None)(
        x
    )


def cnn1d_block(inputs_data: Layer, filters: int, kernel_size: int) -> Layer:
    x = Conv1D(
        filters=filters, kernel_size=kernel_size, use_bias=False, padding="causal"
    )(inputs_data)
    x = BatchNormalization()(x)
    return Activation("relu")(x)


## GRU

The gated recurrent neural network.

LSTM (Long Short Term Memory): LSTM has three gates (input, output and forget gate)

GRU (Gated Recurring Units): GRU has two gates (reset and update gate).

GRU couples forget as well as input gates.
GRU use less training parameters and therefore use less memory, execute faster and train faster than LSTM's.

In [ ]:
def get_model_gru(
    window_size: int, features: int, other_features: int, output_slots: int
) -> Model:
    LOGGER.info("Get model GRU")
    # define model
    inputs1 = Input(shape=(window_size, features))
    x = Bidirectional(
        GRU(
            128,
            input_shape=(window_size, features),
            return_sequences=True,
        )
    )(inputs1)
    x = GRU(
        256,
        input_shape=(window_size, features),
        return_sequences=True,
    )(x)
    x = GRU(
        512,
        input_shape=(window_size, features),
        return_sequences=False,
    )(x)
    flat1 = Flatten()(x)

    x = dense_block(flat1, 1024)
    x = dense_block(x, 1024)
    x = dense_block(x, 512)
    x = Dropout(0.2)(x)

    inputs2 = Input(shape=(other_features,))
    flat2 = Dense(72, activation="relu")(inputs2)
    merged = concatenate([x, flat2], name="merged")
    x = dense_block(merged, 512)

    outputs = Dense(output_slots, activation="tanh", name="final_output")(x)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs, name="gru")


## CNN1D

A multi-headed CNN.
This uses three different kernel sizes in parallel to extract, small, medium and large features

In [ ]:
def get_model_cnn1d(
    window_size: int, features: int, other_features: int, output_slots: int
) -> Model:
    LOGGER.info("Get model CNN1d")
    # define model
    inputs1 = Input(shape=(window_size, features))

    # Head 1
    x = cnn1d_block(inputs1, 32, 3)
    x = cnn1d_block(x, 32, 3)
    x = MaxPooling1D(pool_size=2)(x)

    x = cnn1d_block(x, 64, 3)
    x = cnn1d_block(x, 128, 3)
    x = MaxPooling1D(pool_size=2)(x)
    flat1 = Flatten()(x)

    # Head 2
    x = cnn1d_block(inputs1, 32, 5)
    x = cnn1d_block(x, 32, 5)
    x = MaxPooling1D(pool_size=2)(x)

    x = cnn1d_block(x, 64, 5)
    x = cnn1d_block(x, 128, 5)
    x = MaxPooling1D(pool_size=2)(x)
    flat2 = Flatten()(x)

    # head 3
    x = cnn1d_block(inputs1, 32, 9)
    x = cnn1d_block(x, 32, 9)
    x = MaxPooling1D(pool_size=2)(x)

    x = cnn1d_block(x, 64, 9)
    x = cnn1d_block(x, 128, 9)
    x = MaxPooling1D(pool_size=2)(x)
    flat3 = Flatten()(x)

    merged = concatenate([flat1, flat2, flat3], name="merged1")

    x = dense_block(merged, 4096)
    x = Dropout(0.2)(x)
    x = dense_block(x, 1024)
    x = dense_block(x, 512)
    x = Dropout(0.2)(x)

    inputs2 = Input(shape=(other_features,))
    flat4 = Dense(72, activation="relu")(inputs2)
    merged = concatenate([x, flat4], name="merged2")
    x = dense_block(merged, 512)

    outputs = Dense(output_slots, activation="tanh", name="final_output")(x)

        model = Model(
        inputs=[inputs1, inputs2],
        outputs=outputs,
        name="cnn1d",
    )



## ResNet

In [ ]:
def get_model_resnet1d(
    window_size: int, features: int, other_features: int, output_slots: int
) -> Model:
    LOGGER.info("Get model resnet")
    # define model
    inputs1 = Input(shape=(window_size, features))
    kernel_size = 6
    x = Conv1D(filters=128, kernel_size=kernel_size, padding="causal")(inputs1)
    x = BatchNormalization()(x)

    # Block 1
    x = cnn_residual_block(x, 128, kernel_size)

    # 1/2 the size
    x = Conv1D(
        filters=256, kernel_size=kernel_size, strides=2, use_bias=True, padding="causal"
    )(x)

    # Block 2
    x = cnn_residual_block(x, 256, kernel_size)

    # 1/2 the size
    x = Conv1D(
        filters=512, kernel_size=kernel_size, strides=2, use_bias=True, padding="causal"
    )(x)
    flat1 = Flatten()(x)

    x = dense_block(flat1, 8192)
    x = dense_block(x, 2048)
    x = dense_block(x, 512)
    x = Dropout(0.2)(x)

    inputs2 = Input(shape=(other_features,))
    flat2 = Dense(72, activation="relu")(inputs2)
    x = concatenate([x, flat2], name="merged2")
    x = dense_block(x, 512)

    outputs = Dense(output_slots, activation="tanh", name="final_output")(x)

    model = Model(
        inputs=[inputs1, inputs2],
        outputs=outputs,
        name="ResNet",
    )


## Dense

In [ ]:
def get_model_dense(
    window_size: int, features: int, other_features: int, output_slots: int
) -> Model:
    LOGGER.info("Get model Dense")
    # define model
    inputs1 = Input(shape=(window_size, features))
    x = Flatten()(inputs1)
    x = dense_block(x, 8192)
    x = dense_block(x, 4096)
    x = Dropout(0.2)(x)
    x = dense_block(x, 2048)
    x = dense_block(x, 1024)
    x = Dropout(0.2)(x)

    # Other inputs
    inputs2 = Input(shape=(other_features,))
    flat2 = Dense(72, activation="relu")(inputs2)
    merged = concatenate([x, flat2], name="merged2")
    x = dense_block(merged, 512)

    outputs = Dense(output_slots, activation="tanh", name="final_output")(x)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs, name="dense")



## Ensemble

In [ ]:
def get_ensemble_model(result_dictionary: Dict, dense_nodes: int) -> Model:
    LOGGER.info("Get model Ensemble")
    head_inputs = []
    merge_inputs = []
    input_data = next(iter(result_dictionary.values()))
    shapes = input_data.shape[1]

    # define model starting with multiheaded input
    for key in SUB_MODELS:
        if key in result_dictionary:
            inputs = Input(shape=(shapes,), name=f"inputs-{key}")
            head_inputs.append(inputs)
            x = dense_block(inputs, shapes)
            x = dense_block(x, shapes)
            merge_inputs.append(x)

    # Merge
    merged = concatenate(merge_inputs)
    x = dense_block(merged, dense_nodes)
    x = dense_block(x, dense_nodes)
    x = dense_block(x, dense_nodes)

    outputs = Dense(shapes, activation="tanh", name="final_output")(x)

    model = Model(inputs=head_inputs, outputs=outputs)


## S3 options

In [ ]:
class S3ModelCheckpoint(Callback):
    """Callback to checkpoint code to S3 if needed"""

    def __init__(
        self,
        s3_bucket: str,
        filename: PosixS3Name,
        pause: int,
    ):
        super().__init__()

        if s3_bucket is not None:
            self._s3_copy = S3Helper(s3_bucket)
            self._filename = filename
            self._last_copy = self._mtime = time()
            self._pause = pause
        else:
            self._s3_copy = None

    def on_epoch_end(self, epoch, logs=None):
        if self._s3_copy is not None and exists(self._filename.posix_name):
            new_mtime = getmtime(self._filename.posix_name)

            # The file has changed
            if new_mtime <= self._mtime:
                return

            if time() >= self._last_copy + self._pause:
                self._s3_copy.copy_to_s3(
                    self._filename.posix_name, self._filename.s3_name
                )

                # Update the last copy time
                self._last_copy = time()

                # Update the file mtime
                self._mtime = new_mtime


# Running in AWS

Running in AWS can get expensive

```
#!/bin/bash

if [ -z "$1" ]; then
    echo -e "\nPlease call '$0 <month>' to run this command\n"
    exit 1
fi

# Put a kill switch in
sudo shutdown -h 1080

# Run the code
cd /home/ubuntu/ML-Chevron-2020/src
git pull

pipenv run python3 main.py train-spectra --yaml-tag=96-48 --months=$1

pipenv run python3 main.py ensemble-spectra --yaml-tag=ensemble_spectra --months=$1

echo Training complete

sudo shutdown -h now
```

# Running in BoM

BoM runs on the NCI.
So jobs are submitted as SLURM.